In [13]:
import cv2
import os
import numpy as np
import pandas as pd
import mediapipe as mp
import warnings
warnings.filterwarnings('ignore')

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

order_number = '12'
   
npy_save_path = r'H:\1.media_pipe\수어 영상\npy' \
                + '\\' + order_number + '_real_word_npy_30f_F_U_D_fixed'
os.makedirs(npy_save_path, exist_ok=True)
error_path = r'H:\1.media_pipe\수어 영상\npy\error'
error_txt = open(error_path + '\\' + npy_save_path.split('\\')[-1] + '.txt', 'a')
video_path = r'H:\1.media_pipe\수어 영상\1.Training' \
            + '\\' + order_number
Label = pd.read_csv(r'H:\1.media_pipe\수어 영상\Label' \
                    + '\\' + order_number + '_real_word_label.csv')

video_list = os.listdir(video_path)

In [14]:
l_F = Label[Label['name'].str.contains('_F.mp4')]
l_U = Label[Label['name'].str.contains('_U.mp4')]
l_D = Label[Label['name'].str.contains('_D.mp4')]

l_F = l_F.append(l_U)
l_F = l_F.append(l_D)
l_F.head()

label_cl_4 = pd.read_csv('./Label_classification_4.csv', encoding='CP949')

label_cl_4_list = []
label_cl_4_list.append(label_cl_4['동사'].dropna().to_list())
label_cl_4_list.append(label_cl_4['명사'].dropna().to_list())
label_cl_4_list.append(label_cl_4['형용사'].dropna().to_list())

label_cl_4_list = sum(label_cl_4_list, [])
len(label_cl_4_list)

new_df = pd.DataFrame()

for a in label_cl_4_list:
    tmp = l_F[l_F['label'] == a]
    new_df = new_df.append(tmp)


new_video_list = new_df['name'].sort_values().to_list()
len(new_video_list)

print('## origin video_list_len : ', len(video_list))
print('## new_video_list_len : ', len(new_video_list))
print('## video[0] : ', new_video_list[0])
print('## save npy folder : ', npy_save_path.split('\\')[-1])
print('\n## label data')
print(Label.head())

## origin video_list_len :  15000
## new_video_list_len :  489
## video[0] :  NIA_SL_WORD0009_REAL12_D.mp4
## save npy folder :  12_real_word_npy_30f_F_U_D_fixed

## label data
                           name  duration  start    end label
0  NIA_SL_WORD0001_REAL12_D.mp4       4.7   1.88  3.717    고민
1  NIA_SL_WORD0001_REAL12_F.mp4       4.7   1.88  3.717    고민
2  NIA_SL_WORD0001_REAL12_L.mp4       4.7   1.88  3.717    고민
3  NIA_SL_WORD0001_REAL12_R.mp4       4.7   1.88  3.717    고민
4  NIA_SL_WORD0001_REAL12_U.mp4       4.7   1.88  3.717    고민


In [15]:
import numpy as np
import cv2
import mediapipe as mp
from tqdm import tqdm

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

error_count = 0

for k, j in enumerate(tqdm(new_video_list)):

    # if k == 1: # test
    #     break
    # j = 'NIA_SL_WORD1384_REAL04_F.mp4' #test

    cap = cv2.VideoCapture(os.path.join(video_path, j))

    right_keypoints = []
    left_keypoints = []
    both_hands_list = []

    frame_count = 0

    # start = int(Label.iloc[k]['start'] * 30)
    # end = int(Label.iloc[k]['end'] * 30)
    
    # duration = int(Label.iloc[k]['duration'] * 30)
    duration = cap.get(cv2.CAP_PROP_FRAME_COUNT)

    
    
    duration_division = duration // 30
    
    duration_division_list = [duration_division * i for i in range(31)]

    # print('## duration : ', duration)
    # print('## duration_d : ', duration_division)
    
    with mp_hands.Hands(
        model_complexity = 0,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands:
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                break

            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = hands.process(image)
            
            right_keypoint = np.zeros((21, 3))
            left_keypoint = np.zeros((21, 3))

            if results.multi_hand_landmarks is not None and frame_count != 0:
                if len(results.multi_hand_landmarks) == 2:
                    for i in range(21):
                        if results.multi_hand_landmarks[0].landmark[0].x > results.multi_hand_landmarks[1].landmark[0].x:
                            right_keypoint[i, 0] = results.multi_hand_landmarks[0].landmark[i].x
                            right_keypoint[i, 1] = results.multi_hand_landmarks[0].landmark[i].y
                            right_keypoint[i, 2] = results.multi_hand_landmarks[0].landmark[i].z

                            left_keypoint[i, 0] = results.multi_hand_landmarks[1].landmark[i].x
                            left_keypoint[i, 1] = results.multi_hand_landmarks[1].landmark[i].y
                            left_keypoint[i, 2] = results.multi_hand_landmarks[1].landmark[i].z
                        else:
                            right_keypoint[i, 0] = results.multi_hand_landmarks[1].landmark[i].x
                            right_keypoint[i, 1] = results.multi_hand_landmarks[1].landmark[i].y
                            right_keypoint[i, 2] = results.multi_hand_landmarks[1].landmark[i].z

                            left_keypoint[i, 0] = results.multi_hand_landmarks[0].landmark[i].x
                            left_keypoint[i, 1] = results.multi_hand_landmarks[0].landmark[i].y
                            left_keypoint[i, 2] = results.multi_hand_landmarks[0].landmark[i].z


                right_keypoints.append(right_keypoint)
                left_keypoints.append(left_keypoint)
            

            if frame_count in duration_division_list and frame_count != 0:
                # print('frame_count', frame_count)
                
                # print('# ', np.sum(right_keypoints, axis=1))
                right_keypoints = np.array(right_keypoints)
                left_keypoints = np.array(left_keypoints)

                # print(str(right_keypoints.shape))
                # print()
                try:
                    r_zero_count = sum(np.sum(right_keypoints, axis=1)[:, 0] == 0)
                    l_zero_count = sum(np.sum(left_keypoints, axis=1)[:, 0] == 0)
                except:
                    print('cannot find hand on 4 frames')
                    break

                right_sum = np.sum(right_keypoints, axis=0)
                left_sum = np.sum(left_keypoints, axis=0)

                right_mean = right_sum / (duration_division - r_zero_count)
                left_mean = left_sum / (duration_division - l_zero_count)

                # right_mean = np.mean(right_keypoints, axis=0)
                # # right_std = np.std(right_keypoints, axis=0)
                # left_mean = np.mean(left_keypoints, axis=0)
                # left_std = np.std(left_keypoints, axis=0)
                # both_hands = np.concatenate((right_mean, left_mean, right_std, left_std))
                
                # both_hands = np.concatenate((right_mean, left_mean))
                
                try:
                    both_hands = np.concatenate((right_mean, left_mean))
                    # both_hands = np.concatenate((right_mean, left_mean, right_std, left_std))
                except:
                    print('both_error')
                    break
                
                both_hands_list.append(both_hands)

                right_keypoints = []
                left_keypoints = []
                
            frame_count += 1
        
        try:
            save_file = np.array(both_hands_list)
            save_file = save_file.reshape(3780)
            np.save(npy_save_path + f'\\{j[:-4]}.npy', save_file)
        except:
            print(j)
            error_txt.write(j + '\n')
            error_count += 1
            continue

cap.release()
error_txt.close()
print()
print('## Error Count : ', error_count)
print('## Done')

  0%|          | 0/489 [00:00<?, ?it/s]

  6%|▋         | 31/489 [01:40<24:22,  3.19s/it]

cannot find hand on 4 frames
NIA_SL_WORD0694_REAL12_D.mp4


  7%|▋         | 34/489 [01:49<24:13,  3.19s/it]

NIA_SL_WORD0702_REAL12_D.mp4


...

NIA_SL_WORD2132_REAL12_F.mp4


 77%|███████▋  | 375/489 [16:46<05:37,  2.96s/it]

NIA_SL_WORD2132_REAL12_U.mp4


100%|██████████| 489/489 [21:44<00:00,  2.67s/it]


## Error Count :  17
## Done


## 데이터 추출 중 문제점 기록
1. 4프레임 동안 손을 찾지 못하여 right_hands 의 shape이 (0,) 이 되는 문제
2. 4프레임 중 양 손을 찾지 못해 스킵된 채 진행되어 (duration_division, 21, 3)의 사이즈가 아닌 데이터가 존재
 2번 예시 정상 데이터 == (4, 21, 3) / 오류 데이터 == (3, 21, 3) or (2, 21, 3)이 존재
3. nan.....
3-1. nan 원인 : 0을 0으로 나누는 상황으로 추측